<a href="https://colab.research.google.com/github/vladgap/Various/blob/ver1/MLN-101121_article_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Perceptron -- neuron with an activation function.
Inputs and their weights are numbered 0 to(n-1), bias is n

In [1]:
import numpy as np

# Single Neuron

In [2]:
class Neuron:
    """A single neuron with an activation function.
       Attributes:
          inputs: The number of inputs in the perceptron, not counting the bias.
          bias:   The bias term. By defaul it's 1.0.
          activ:  The activation function: linear (default), relu, sigmoid."""

    def __init__(self, inputs, bias = 1.0, activ = 'linear'):
        """Return a new Perceptron object with the specified number of inputs (+1 for the bias) and random initial weights.""" 
        self.weights = (np.random.rand(inputs+1) * 2) - 1 
        self.bias = bias
        self.activ = activ

    def run(self, x):
        """Run the perceptron. x is a python list with the input values."""
        sum = np.dot(np.append(x,self.bias),self.weights)
        if self.activ == 'linear':
          return sum
        if self.activ == 'sigmoid':
          return self.sigmoid(sum)
        if self.activ == 'relu':
          return self.relu(sum)

    def set_weights(self, w_init):
        """Overrides the np.random.rand() weights and the bias weight."""
        # w_init is a list of floats. Organize it as you'd like.
        self.weights=np.array(w_init)

    def set_activ(self, activ):
        """Overrides the 'linear' activation function."""
        self.activ = activ

    def sigmoid(self, x):
        # return the output of the sigmoid function applied to x
        return 1/(1+np.exp(-x))
    
    def relu(self, x):
        # return the output of the relu function applied to x
        if x >= 0:
          return x
        return 0

In [3]:
neu=Neuron(inputs=2)
neu.set_weights([10,10,-15]) 
# neu.set_activ('sigmoid')
neu.run([1,1])

5.0

## AND gate

In [4]:
neuron = Neuron(inputs=2, activ='sigmoid')
neuron.set_weights([10,10,-15]) #AND gate

print("AND Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))


AND Gate:
0 0 = 0.0000003059
0 1 = 0.0066928509
1 0 = 0.0066928509
1 1 = 0.9933071491


In [5]:
#proverka
# a=[]
# a.append(Perceptron(inputs=2))
# a[1].weights

## OR gate

In [6]:
neuron = Neuron(inputs=2, activ='sigmoid')
neuron.set_weights([10,10,-5]) #OR gate

print("OR Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))


OR Gate:
0 0 = 0.0066928509
0 1 = 0.9933071491
1 0 = 0.9933071491
1 1 = 0.9999996941


## NAND gate

In [7]:
neuron = Neuron(inputs=2, activ='sigmoid')
neuron.set_weights([-10,-10,15]) #NAND gate

print("NAND Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))

NAND Gate:
0 0 = 0.9999996941
0 1 = 0.9933071491
1 0 = 0.9933071491
1 1 = 0.0066928509


# Multilayer neuron

In [46]:
class MultiLayerNeuron:     
    """A multilayer neuron class that uses the Neuron class above.
       Builds a list of neurons with the specific activation function.
       The activation function may be modified later using the set_activ method.
       For example: mln.network[layer][neuron].set_activ('linear').
       Attributes:
          layers:  A list with the number of neurons per layer. Including the input (first) and the output (last) layers.
          bias:    The bias term. The same bias is used for all neurons.
          eta:     The learning rate.
          activ:   The activation function: linear (default), relu, sigmoid."""

    def __init__(self, layers, bias = 1.0, eta = 0.5, activ='linear'):
        """Return a new MLP object with the specified parameters.""" 
        self.layers = np.array(layers,dtype=object)
        self.bias = bias
        self.eta = eta
        self.network = [] # The list of lists of neurons (perceptrons).
        self.values = []  # The list of lists of neurons' (perceptrons') output values.
        self.d = []       # The list of lists of error terms (lowercase deltas)
        self.activ = activ

        # 2 nested loops to create neurons layer by layer
        for i in range(len(self.layers)): # outer loop iterates on each layer
            self.values.append([]) #The new list of values will be filled with zeros, for every neuron in the layer. 
            self.values[i] = [0.0 for j in range(self.layers[i])]
            self.d.append([])
            self.d[i] = [0.0 for j in range(self.layers[i])]                        
            self.network.append([])
            if i > 0:      #network[0] is the input layer, so it has no neurons
                for j in range(self.layers[i]): # inner loop iterates on each neuron in a layer
                    neur=Neuron(inputs = self.layers[i-1], bias = self.bias, activ = self.activ) # 
                    self.network[i].append(neur) # adding j perceptrons
        
        self.network = np.array([np.array(x) for x in self.network],dtype=object) #transforms list of lists to numpy array
        self.values = np.array([np.array(x) for x in self.values],dtype=object)
        self.d = np.array([np.array(x) for x in self.d],dtype=object)

    def set_weights(self, w_init): # set_weights of the MultiLayer class
        """Set the weights. 
           w_init is a list of lists with the weights for all but the input layer."""
        for i in range(len(w_init)):
            for j in range(len(w_init[i])):
                self.network[i+1][j].set_weights(w_init[i][j]) # set_weights for each perceptron i

    def set_activ(self, activ):
        """Set the activation function to every neurons."""
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):
                self.network[i][j].set_activ(activ) # set_activ for each neuron
    
    def set_output_activ(self, activ):
        """Set the activation function to the last (output) neurons."""
        i = len(self.network)-1
        for j in range(self.layers[i]):
            self.network[i][j].set_activ(activ) 

    def printWeights(self):
        """Displays a summary of weights and activation functions per layer and neuron."""
        print()
        print('Layer 0 is the Input Layer')
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):
                print("Layer",i,"Neuron",j,":",self.network[i][j].weights,self.network[i][j].activ)
        print()

    def run(self, x):
        """Feed a sample x into the MultiLayer Neuron."""
        x = np.array(x,dtype=object)
        self.values[0] = x
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):  
                self.values[i][j] = self.network[i][j].run(self.values[i-1]) #runs preceptrons with the previous outputs
        return self.values[-1]

    def bp_classif(self, x, y):
        """Run a single (x,y) pair with the backpropagation algorithm - Gradient Descent.
        Uses the derivative of the sigmoid function."""
        x = np.array(x,dtype=object)
        y = np.array(y,dtype=object)
        # STEP 1: Feed a sample to the network 
        outputs = self.run(x)
        # STEP 2: Calculate the MSE
        error = 2*(y - outputs) # A list of outputs
        MSE = sum( error ** 2) / self.layers[-1] 
        # ∂MSE/∂weight=∂MSE/∂output*∂output/∂weight
        # STEP 3: Calculate the OUTPUT error terms
        # ∂MSE/∂output -- depends on neuron's activation function
        self.d[-1] = outputs * (1 - outputs) * (error) # derivative of the SIGMOID function 
        # STEP 4: Calculate the error term of EACH UNIT on each layer
        for i in reversed(range(1,len(self.network)-1)):
            for h in range(len(self.network[i])):
                fwd_error = 0.0
                for k in range(self.layers[i+1]): 
                    fwd_error += self.network[i+1][k].weights[h] * self.d[i+1][k]               
                self.d[i][h] = self.values[i][h] * (1-self.values[i][h]) * fwd_error # derivative of the SIGMOID function
        # STEPS 5 & 6: Calculate the deltas and update the weights
        for i in range(1,len(self.network)): # runs on layers
            for j in range(self.layers[i]): # runs on neurons
                for k in range(self.layers[i-1]+1): # runs on inputs. +1 for bias
                    if k==self.layers[i-1]:
                        delta = self.eta * self.d[i][j] * self.bias
                    else:
                        delta = self.eta * self.d[i][j] * self.values[i-1][k] # applying the delta rule
                    self.network[i][j].weights[k] += delta
        return MSE

    def bp_regres(self, x, y):
        """Run a single (x,y) pair with the backpropagation algorithm - Gradient Descent.
        Uses the derivative of the linear function."""
        x = np.array(x,dtype=object)
        y = np.array(y,dtype=object)
        # STEP 1: Feed a sample to the network 
        outputs = self.run(x)
        # STEP 2: Calculate the MSE
        error = 2*(y - outputs) # A list of outputs
        MSE = sum( error ** 2) / self.layers[-1] 
        # STEP 3: Calculate the OUTPUT error terms
        self.d[-1] = (error) 
        # STEP 4: Calculate the error term of EACH UNIT on each layer
        for i in reversed(range(1,len(self.network)-1)):
            for h in range(len(self.network[i])):
                fwd_error = 0.0
                for k in range(self.layers[i+1]): 
                    fwd_error += self.network[i+1][k].weights[h] * self.d[i+1][k]               
                self.d[i][h] = fwd_error 
        # STEPS 5 & 6: Calculate the deltas and update the weights
        for i in range(1,len(self.network)): # runs on layers
            for j in range(self.layers[i]): # runs on neurons
                for k in range(self.layers[i-1]+1): # runs on inputs. +1 for bias
                    if k==self.layers[i-1]:
                        delta = self.eta * self.d[i][j] * self.bias
                    else:
                        delta = self.eta * self.d[i][j] * self.values[i-1][k] # applying the delta rule
                    self.network[i][j].weights[k] += delta
        return MSE

    def deriv(self, value, i, j=0):
        if self.network[i][j].activ == 'linear':
          # print ('lin')
          return value
        if self.network[i][j].activ == 'sigmoid':
          # print ('sig')
          return value*(1-value)
        if self.network[i][j].activ == 'relu':
          if value > 0:
            # print ('re>')
            return value
          else:
            # print ('re<')
            return 0

    def bp_general(self, x, y):
        """Run a single (x,y) pair with the backpropagation algorithm - Gradient Descent.
        Uses the derivative according each neuron's activation function."""
        x = np.array(x,dtype=object)
        y = np.array(y,dtype=object)
        # STEP 1: Feed a sample to the network 
        outputs = self.run(x)
        # STEP 2: Calculate the MSE
        error = 2*(y - outputs) # A list of outputs
        print ('error', error)
        MSE = sum( error ** 2) / self.layers[-1] 
        # ∂MSE/∂weight=∂MSE/∂output*∂output/∂weight
        # STEP 3: Calculate the OUTPUT error terms
        # ∂MSE/∂output -- depends on neuron's activation function
        for j in range (len(outputs)):
            self.d[-1][j] = self.deriv(outputs[j], len(self.network)-1) * error
            print ('len', self.network[len(self.network)-1][0].activ)
            print ('deriv', self.deriv(outputs[j], len(self.network)-1))
            print ('d last',self.d[-1][j])
        # STEP 4: Calculate the error term of EACH UNIT on each layer
        for i in reversed(range(1,len(self.network)-1)):
            for h in range(len(self.network[i])):
                fwd_error = 0.0
                for k in range(self.layers[i+1]): 
                    fwd_error += self.network[i+1][k].weights[h] * self.d[i+1][k] 
                    print ('fwd_error', fwd_error)              
                self.d[i][h] = self.deriv(self.values[i][h], i, h) * fwd_error
        # STEPS 5 & 6: Calculate the deltas and update the weights
        for i in range(1,len(self.network)): # runs on layers
            for j in range(self.layers[i]): # runs on neurons
                for k in range(self.layers[i-1]+1): # runs on inputs. +1 for bias
                    # output=sum(weight*value)+bias*bias_weight
                    if k==self.layers[i-1]:
                        # ∂output/∂bias_weight=bias
                        delta = self.eta * self.d[i][j] * self.bias
                    else:
                        # ∂output/∂weight=value
                        delta = self.eta * self.d[i][j] * self.values[i-1][k] 
                    self.network[i][j].weights[k] += delta # applying the delta rule
        return MSE

## XOR gate=(OR+NAND)+AND

In [9]:
#test code
mln1 = MultiLayerNeuron(layers=[2,2,1])  #mln1
mln1.set_weights([[[-10,-10,15],[15,15,-10]],[[10,10,-15]]])
mln1.set_activ('sigmoid') #linear is by default

mln1.printWeights()
print("XOR Gate:")
print ("0 0 = {0:.10f}".format(mln1.run([0,0])[0]))
print ("0 1 = {0:.10f}".format(mln1.run([0,1])[0]))
print ("1 0 = {0:.10f}".format(mln1.run([1,0])[0]))
print ("1 1 = {0:.10f}".format(mln1.run([1,1])[0]))


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-10 -10  15] sigmoid
Layer 1 Neuron 1 : [ 15  15 -10] sigmoid
Layer 2 Neuron 0 : [ 10  10 -15] sigmoid

XOR Gate:
0 0 = 0.0066958493
0 1 = 0.9923558642
1 0 = 0.9923558642
1 1 = 0.0071528098


## Training 

In [10]:
mln2 = MultiLayerNeuron(layers=[2,2,1])
mln2.set_activ('sigmoid') #linear is by default
print("\nTraining Neural Network as an XOR Gate...\n")
for i in range(5000):
    MSE = 0.0
    MSE += mln2.bp_classif([0,0],[0])
    MSE += mln2.bp_classif([0,1],[1])
    MSE += mln2.bp_classif([1,0],[1])
    MSE += mln2.bp_classif([1,1],[0])
    MSE = MSE / 4
    if(i%200 == 0):
        print (MSE)

mln2.printWeights()
    
print("XOR Gate:")
print ("0 0 = {0:.10f}".format(mln2.run([0,0])[0]))
print ("0 1 = {0:.10f}".format(mln2.run([0,1])[0]))
print ("1 0 = {0:.10f}".format(mln2.run([1,0])[0]))
print ("1 1 = {0:.10f}".format(mln2.run([1,1])[0]))


Training Neural Network as an XOR Gate...

0.2718627885118745
0.2616943470604117
0.26096741423546416
0.26020749867912646
0.2593435770704745
0.25783755667520303
0.25133108656050257
0.198180171154835
0.04560962109455799
0.015938788699405158
0.008758173178449988
0.005845778255263611
0.004322644799038431
0.0034015911898457904
0.0027902695710590618
0.002357468737226992
0.0020362203499594863
0.001789012632104645
0.0015932995859940357
0.0014347605776547747
0.0013038858499775235
0.0011941251100407823
0.0011008263022826085
0.0010205985149352842
0.000950914245682344

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-5.61348977 -5.62167426  2.13157724] sigmoid
Layer 1 Neuron 1 : [ 4.78467716  4.78586318 -7.44384055] sigmoid
Layer 2 Neuron 0 : [-8.76131309 -8.68586157  4.26600216] sigmoid

XOR Gate:
0 0 = 0.0273550563
0 1 = 0.9688650041
1 0 = 0.9688216238
1 1 = 0.0294393808


# Example1 -- weights

In [11]:
print ("1 1 =",mln1.run([1,1]))
print (mln1.network[1][0].weights) # network is list of lists of perceptrons. Each has attribute "weights"

1 1 = [0.00715281]
[-10 -10  15]


In [12]:
mln1=MultiLayerNeuron(layers=[2,2,1])
mln1.set_activ('relu')
mln1.printWeights()

mln1.set_output_activ('sigmoid') # setting the output activ func
mln1.printWeights()

mln1.network[1][0].set_activ('linear') # changing specific activ func
mln1.printWeights()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-0.85603414 -0.98960953 -0.29155378] relu
Layer 1 Neuron 1 : [ 0.90784257 -0.91450963 -0.32838058] relu
Layer 2 Neuron 0 : [ 0.95558115  0.77766704 -0.47094694] relu


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-0.85603414 -0.98960953 -0.29155378] relu
Layer 1 Neuron 1 : [ 0.90784257 -0.91450963 -0.32838058] relu
Layer 2 Neuron 0 : [ 0.95558115  0.77766704 -0.47094694] sigmoid


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-0.85603414 -0.98960953 -0.29155378] linear
Layer 1 Neuron 1 : [ 0.90784257 -0.91450963 -0.32838058] relu
Layer 2 Neuron 0 : [ 0.95558115  0.77766704 -0.47094694] sigmoid



# Example 2 -- sigmoid, bp_general, XOR

In [13]:
mln2 = MultiLayerNeuron(layers=[2,2,1], eta=0.5)
mln2.set_activ('sigmoid') #linear is by default
print("\nTraining Neural Network as an XOR Gate...\n")
for i in range(5000):
    MSE = 0.0
    MSE += mln2.bp_general([0,0],[0])
    MSE += mln2.bp_general([0,1],[1])
    MSE += mln2.bp_general([1,0],[1])
    MSE += mln2.bp_general([1,1],[0])
    MSE = MSE / 4
    if(i%200 == 0):
        print (MSE)

mln2.printWeights()
    
print("XOR Gate:")
print ("0 0 = {0:.10f}".format(mln2.run([0,0])[0]))
print ("0 1 = {0:.10f}".format(mln2.run([0,1])[0]))
print ("1 0 = {0:.10f}".format(mln2.run([1,0])[0]))
print ("1 1 = {0:.10f}".format(mln2.run([1,1])[0]))


Training Neural Network as an XOR Gate...

0.28762557870012756
0.26063099628961894
0.25936263355810446
0.25628226563399437
0.23388953702633014
0.18408738208882963
0.13569821124079304
0.037153480177998466
0.013992362891620412
0.007910801556913422
0.005364773757151355
0.004008310515854911
0.0031776213797063404
0.0026211252550958925
0.0022242914819338134
0.001928032332777406
0.0016989658782186237
0.0015168869594000076
0.001368885871777498
0.001246346444263098
0.0011433078444255925
0.001055520347139098
0.0009798757197430768
0.0009140500832148515
0.0008562724596224377

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-6.16400432 -6.07890496  2.4323488 ] sigmoid
Layer 1 Neuron 1 : [-4.46415029 -4.44658589  6.6127329 ] sigmoid
Layer 2 Neuron 0 : [-9.11529869  8.89904297 -4.1687117 ] sigmoid

XOR Gate:
0 0 = 0.0250638748
0 1 = 0.9729733626
1 0 = 0.9730767147
1 1 = 0.0336721574


# Example 3 -- linear activs, bp_regres

In [14]:
mln3 = MultiLayerNeuron(layers=[3,1], eta=0.3)
# mln3.set_weights([[[4.9,-0.9,2.1,0.2]]])
# mln3.printWeights()
print("\nTraining Neural Network...\n")
for i in range(400):
    MSE = 0.0
    MSE += mln3.bp_regres([0.7759,	0.1104,	0.9977,],[5.764286995])
    MSE += mln3.bp_regres([0.9692,	0.6961,	0.8483,],[5.84646758])
    MSE += mln3.bp_regres([0.0265,	0.399,	0.5375,],[0.808633075])
    MSE += mln3.bp_regres([0.7694,	0.5051,	0.2542,],[3.850298589])
    MSE = MSE / 4
    if(i%200 == 0):
        print (MSE)

mln3.printWeights()




Training Neural Network...

10.16174059255133
2.3016137465785132e-09

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 4.99973370e+00 -9.99625031e-01  1.99991680e+00  3.52339498e-05] linear



# Example 4 -- sigmoid, bp_regres

In [15]:
# generating output values
mln4 = MultiLayerNeuron(layers=[3,2,1], eta=0.3)
mln4.set_activ('sigmoid')
mln4.set_output_activ('linear')
mln4.set_weights([[[0,1,-2,1],[1,-2,0,1]],[[-1,2,3]]])
mln4.printWeights()

y4=[]
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('original values:',y4)
print ()

# training model and predicting values
mln4 = MultiLayerNeuron(layers=[3,2,1], eta=0.3)
mln4.set_activ('sigmoid')
mln4.set_output_activ('linear')

for i in range(1000):
    MSE = 0.0
    MSE += mln4.bp_regres([2,1,0],y4[0])
    MSE += mln4.bp_regres([-1,0,1],y4[1])
    MSE += mln4.bp_regres([0,0,0],y4[2])
    MSE += mln4.bp_regres([1,1,0],y4[3])
    MSE += mln4.bp_regres([-1,-1,-1],y4[4])
    MSE += mln4.bp_regres([-1,1,-1],y4[5])
    MSE = MSE / 6
    if(i%200 == 0):
        print ('MSE=',MSE)
mln4.printWeights()

y4=[]
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('predicted values:',y4)
print ()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0  1 -2  1] sigmoid
Layer 1 Neuron 1 : [ 1 -2  0  1] sigmoid
Layer 2 Neuron 0 : [-1  2  3] linear



'original values:'

[3.5813200792821274,
 3.731058578630005,
 3.731058578630005,
 3.119202922022118,
 3.880797077977882,
 2.2563920540063265]


MSE= 2.3696344051105975
MSE= 0.0009200888444749596
MSE= 0.014857977578680334
MSE= 0.1459825105580136
MSE= 0.224887053180854

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 296.11613627 -592.20798332  295.99755154   -3.12342151] sigmoid
Layer 1 Neuron 1 : [ 185.76641893 -371.87968197  186.2390767    -2.86282998] sigmoid
Layer 2 Neuron 0 : [4.85487037 4.64623143 2.94382663] linear



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: RuntimeWarning: overflow encountered in exp


'predicted values:'

[3.3335696543000557,
 3.5163793947052575,
 3.399464504160843,
 2.9438266268414877,
 3.390516117028497,
 2.9438266268414877]

# Example 5 -- various, bp_general

In [16]:
# generating output values
mln4 = MultiLayerNeuron(layers=[3,2,1,1])
mln4.set_activ('relu')
mln4.network[2][0].set_activ('sigmoid') # changing specific activ func
mln4.set_output_activ('linear')
mln4.set_weights([[[0,1,-2,1],[1,-2,0,1]],[[1,0,1]],[[-1,2]]])
mln4.printWeights()

y4=[]
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('original values:',y4)
print ()

# training model and predicting values
mln4 = MultiLayerNeuron(layers=[3,2,1,1], eta=0.3)
mln4.set_activ('relu')
mln4.network[2][0].set_activ('sigmoid') # changing specific activ func
mln4.set_output_activ('linear')

for i in range(1000):
    MSE = 0.0
    MSE += mln4.bp_general([2,1,0],y4[0])
    MSE += mln4.bp_general([-1,0,1],y4[1])
    MSE += mln4.bp_general([0,0,0],y4[2])
    MSE += mln4.bp_general([1,1,0],y4[3])
    MSE += mln4.bp_general([-1,-1,-1],y4[4])
    MSE += mln4.bp_general([-1,1,-1],y4[5])
    MSE = MSE / 6
    if(i%200 == 0):
        print ('MSE=',MSE)
mln4.printWeights()

y4=[]
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('predicted values:',y4)
print ()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0  1 -2  1] relu
Layer 1 Neuron 1 : [ 1 -2  0  1] relu
Layer 2 Neuron 0 : [1 0 1] sigmoid
Layer 3 Neuron 0 : [-1  2] linear



'original values:'

[1.0474258731775667,
 1.2689414213699952,
 1.1192029220221178,
 1.0474258731775667,
 1.0474258731775667,
 1.0066928509242847]


MSE= 0.043333695766815304
MSE= 0.009860927559103774
MSE= 0.010127426796646353
MSE= 0.010097640196507955
MSE= 0.009958407849206707

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0.43573321 -0.67196075 -0.16241134  0.64563345] relu
Layer 1 Neuron 1 : [ 0.36679907 -0.60027958 -0.86786428 -0.21805261] relu
Layer 2 Neuron 0 : [-0.7659057  -0.01978335  0.39341924] sigmoid
Layer 3 Neuron 0 : [0.37496329 0.84812762] linear



'predicted values:'

[1.0119375733969678,
 1.068728069629125,
 1.0261424589110975,
 1.043090766742718,
 0.996459802893019,
 1.072020276700834]

# Example 6 -- https://mmuratarat.github.io/2020-01-09/backpropagation

In [47]:
# generating output values
mln4 = MultiLayerNeuron(layers=[5,3,1], eta=.5)
mln4.set_activ('sigmoid')
mln4.set_output_activ('linear')
mln4.set_weights([[[.19,.33,.4,.51,.54,.1],[.55,.16,.35,.85,.49,.1],[.76,.97,.7,.85,.57,.1]],[[.1,.03,-.17,.1]]]) # setting initial weights
mln4.printWeights()

y4=[]
y4.append(mln4.run([.5,.1,1,0,0])[0])
# y4.append(mln4.run([.3,.2,0,1,0])[0])
# y4.append(mln4.run([.7,.9,0,0,1])[0])
# y4.append(mln4.run([.8,.1,1,0,0])[0])
display ('original values:',y4)
print ()

# training
for i in range(1):
    MSE = 0.0
    MSE += mln4.bp_general([.5,.1,1,0,0],[0.1])
    # MSE += mln4.bp_general([-1,0,1],y4[1])
    # MSE += mln4.bp_general([0,0,0],y4[2])
    # MSE += mln4.bp_general([1,1,0],y4[3])
    # MSE += mln4.bp_general([-1,-1,-1],y4[4])
    # MSE += mln4.bp_general([-1,1,-1],y4[5])
    # MSE = MSE / 6
    # if(i%200 == 0):
    #     print ('MSE=',MSE)
mln4.printWeights()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [0.19 0.33 0.4  0.51 0.54 0.1 ] sigmoid
Layer 1 Neuron 1 : [0.55 0.16 0.35 0.85 0.49 0.1 ] sigmoid
Layer 1 Neuron 2 : [0.76 0.97 0.7  0.85 0.57 0.1 ] sigmoid
Layer 2 Neuron 0 : [ 0.1   0.03 -0.17  0.1 ] linear



'original values:'

[0.05259044255603488]


error [0.09481911488793025]
len linear
deriv 0.05259044255603488
d last 0.004986579214727767
fwd_error 0.0004986579214727768
fwd_error 0.000149597376441833
fwd_error -0.0008477184665037205

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [0.19002828 0.33000566 0.40005657 0.51       0.54       0.10005657] sigmoid
Layer 1 Neuron 1 : [0.55000818 0.16000164 0.35001635 0.85       0.49       0.10001635] sigmoid
Layer 1 Neuron 2 : [0.75996386 0.96999277 0.69992773 0.85       0.57       0.09992773] sigmoid
Layer 2 Neuron 0 : [ 0.10162571  0.03168849 -0.1680504   0.10249329] linear

